In [1]:
import tensorflow as tf

/home/teoac/solarisTutorial/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/teoac/solarisTutorial/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/teoac/solarisTutorial/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/teoac/solarisTutorial/lib/python3.5/site-packages/tensorflow/

In [2]:
# MNIST 데이터 다운로드를 합니다.
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('dataOneHot/', one_hot=True)

W0806 17:14:41.316258 140354939819776 deprecation.py:323] From <ipython-input-2-d2e182775bb2>:3: read_data_sets (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
W0806 17:14:41.317454 140354939819776 deprecation.py:323] From /home/teoac/solarisTutorial/lib/python3.5/site-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:260: maybe_download (from tensorflow.contrib.learn.python.learn.datasets.base) is deprecated and will be removed in a future version.
Instructions for updating:
Please write your own downloading logic.
W0806 17:14:41.318521 140354939819776 deprecation.py:323] From /home/teoac/solarisTutorial/lib/python3.5/site-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:262: extract_images (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be remove

Extracting dataOneHot/train-images-idx3-ubyte.gz


W0806 17:14:41.560288 140354939819776 deprecation.py:323] From /home/teoac/solarisTutorial/lib/python3.5/site-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:267: extract_labels (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use tf.data to implement this functionality.
W0806 17:14:41.562939 140354939819776 deprecation.py:323] From /home/teoac/solarisTutorial/lib/python3.5/site-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:110: dense_to_one_hot (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use tf.one_hot on tensors.
W0806 17:14:41.608931 140354939819776 deprecation.py:323] From /home/teoac/solarisTutorial/lib/python3.5/site-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:290: DataSet.__init__ (from tensorflow.contrib.learn.python.lea

Extracting dataOneHot/train-labels-idx1-ubyte.gz
Extracting dataOneHot/t10k-images-idx3-ubyte.gz
Extracting dataOneHot/t10k-labels-idx1-ubyte.gz


In [3]:
# 학습을 위한 설정값들을 정의합니다.
learning_rate = 0.001
num_epochs = 1000 # 학습횟수
batch_size = 256 # 배치 개수
display_step = 5 # 손실 함수 출력 주기
input_size = 784
hidden1_size = 256
hidden2_size = 256
output_size = 10
tolerance = 0.001

In [4]:
# 입력값과 출력값을 받기 위한 플레이스홀더를 정의합니다.
x = tf.placeholder(tf.float32, shape=[None,input_size], name='x')
y = tf.placeholder(tf.float32, shape=[None,output_size], name='y')

In [5]:
# ANN 모델을 정의합니다.
def build_ANN(x):
    W1 = tf.Variable(tf.random_normal(shape=[input_size,hidden1_size]), name='W1')
    b1 = tf.Variable(tf.random_normal(shape=[hidden1_size]), name='b1')
    H1_output = tf.nn.relu(tf.matmul(x,W1) + b1, name='L1')
    W2 = tf.Variable(tf.random_normal(shape=[hidden1_size, hidden2_size]), name='W2')
    b2 = tf.Variable(tf.random_normal(shape=[hidden2_size]), name='b2')
    H2_output = tf.nn.relu(tf.matmul(H1_output,W2) + b2, name='L2')
    W_output = tf.Variable(tf.random_normal(shape=[hidden2_size,output_size]), name='Wout')
    b_output = tf.Variable(tf.random_normal(shape=[output_size]), name='bout')
    logits = tf.matmul(H2_output, W_output) + b_output
    
    return logits

In [6]:
# ANN 모델을 선언합니다.
predicted_value = build_ANN(x)

In [7]:
# 손실 함수와 옵티마이저를 정의합니다.
loss_function = tf.nn.softmax_cross_entropy_with_logits_v2(logits=predicted_value, labels=y)
loss = tf.reduce_mean(loss_function)
train_step = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)

In [8]:
# 텐서보드를 위한 요약 정보(scalar)를 정의합니다.
tf.summary.scalar("Loss", loss)

<tf.Tensor 'Loss:0' shape=() dtype=string>

In [9]:
# 세션을 열고 그래프를 실행합니다.
with tf.Session() as sess:
    # 변수들에 초기값을 할당합니다.
    sess.run(tf.global_variables_initializer())
    # 텐서보드 요약정보들을 하나로 합칩니다.
    merged = tf.summary.merge_all()
    # 텐서보드 summary 정보들을 저장할 폴더 경로를 설정합니다.
    tensorboard_writer = tf.summary.FileWriter('./tensorboard_log', sess.graph)
    # 지정된 횟수만큼 최적화를 수행합니다.
    epoch = 0
    average_loss_prev = 0.0
    while epoch < num_epochs:
        average_loss = 0.0
        #전체 배치를 불러옵니다.
        total_batch = int(mnist.train.num_examples/batch_size)
        # 모든 배치들에 대해서 최적화를 수행합니다.
        for i in range(total_batch):
            batch_x, batch_y = mnist.train.next_batch(batch_size)
            # 옵티마이저를 실행해서 파라미터들을 업데이트 합니다.
            _, current_loss = sess.run([train_step, loss], feed_dict={x:batch_x, y:batch_y})
            # 평균 손실을 측정합니다.
            average_loss += current_loss / total_batch
        # 텐서보드에 기록합니다.
        summary = sess.run(merged, feed_dict={x: batch_x, y: batch_y})
        tensorboard_writer.add_summary(summary, epoch)
        # 지정된 epoch마다 학습결과를 출력합니다.
        if epoch%display_step == display_step-1 or epoch == 0:
            print('Epoch: %d, Loss: %f' %((epoch+1), average_loss))
        #수렴했으면 학습종료
        if abs(average_loss_prev - average_loss) < tolerance:
            print('Epoch: %d, Loss: %f' %((epoch+1), average_loss))
            break
        average_loss_prev = average_loss
        epoch += 1
        
    # 테스트 데이터를 이용해서 학습된 모델이 얼마나 정확한지 정확도를 출력합니다.
    correct_prediction = tf.equal(tf.argmax(predicted_value,1), tf.argmax(y,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, 'float'))
    print('Accuracy: %f' %(accuracy.eval(feed_dict={x:mnist.test.images, y:mnist.test.labels})))
    
    # 세션을 종료합니다.
    sess.close()

Epoch: 1, Loss: 308.849482
Epoch: 5, Loss: 25.309010
Epoch: 10, Loss: 9.637833
Epoch: 15, Loss: 3.964542
Epoch: 20, Loss: 1.410634
Epoch: 25, Loss: 0.527363
Epoch: 30, Loss: 0.253234
Epoch: 35, Loss: 0.142652
Epoch: 40, Loss: 0.117087
Epoch: 45, Loss: 0.146814
Epoch: 50, Loss: 0.114887
Epoch: 55, Loss: 0.105624
Epoch: 60, Loss: 0.121314
Epoch: 62, Loss: 0.096998
Accuracy: 0.957300
